In [49]:
import random
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm


class Rules:
    def __init__(self, bj_payout=1.5, hit_soft17=True, allow_surrender=True, double_after_split=True, resplit_aces=True, hit_split_aces=False, max_hands=4):
        self.bj_payout = bj_payout
        self.hit_soft17 = hit_soft17
        self.allow_surrender = allow_surrender
        self.double_after_split = double_after_split
        self.resplit_aces = resplit_aces
        self.hit_split_aces = hit_split_aces
        self.max_hands = max_hands
        
class Counter:
    def __init__(self):
        values = [2,3,4,5,6,7,8,9,10,11,1]
        self.systems = {
            'hilow': dict(zip(values, [1,1,1,1,1,0,0,0,-1,-1,-1]))
        }
        self.counts = {system: 0 for system in self.systems}
#         self.observed = dict(zip(values, [0,0,0,0,0,0,0,0,0,0]))
        
    def count(self, card):
#         self.observed[card] += 1
        for system in self.systems:
            self.counts[system] += self.systems[system][card]
    
    def copy(self):
        counter = Counter()
        counter.counts = self.counts.copy()
#         counter.observed = self.observed.copy()
        return counter

class Deck:
    def __init__(self, rules, num_decks, shuffle=True):
        self.rules = rules
        self.num_decks = num_decks
        self.cards = [2,3,4,5,6,7,8,9,10,10,10,10,11] * 4 * self.num_decks
        if shuffle:
            for i in range(5):
                random.shuffle(self.cards)
        self.counter = Counter()
            
    def deal_card(self, count=True):
        card = self.cards.pop()
        if count:
            self.counter.count(card)
        return card
    
    def count(self, card):
        self.counter.count(card)
    
    def cards_remaining(self):
        return len(self.cards)
        
    def decks_remaining(self):
        return self.cards_remaining()/52.
    
    def true_count(self):
        return {k: v/self.decks_remaining() for k, v in self.counter.counts.items()}
    
    def copy(self):
        deck = Deck(self.rules, self.num_decks, shuffle=False)
        deck.cards = self.cards[:]
        deck.counter = self.counter.copy()
        return deck
    
class Hand:
    def __init__(self, deck, cards=[], splitted=False, is_dealer=False, actionable=True, free_hand=False, hand_num=1):
        self.cards = cards[:]
        self.value = 0
        while len(self.cards) < 2:
            if is_dealer and len(self.cards)==0:
                self.cards.append(deck.deal_card(count=False))
            self.cards.append(deck.deal_card())
            
        self.free_hand = free_hand
        self.free_double = False
        self.hand_num = hand_num
        
        self.is_dealer = is_dealer
        self.push = False
        self.splitted = splitted
        self.actionable = actionable
        self.first_action = len(self.cards)==2
        self.pair = self.first_action and (self.cards[0]%10 == self.cards[1]%10) and (hand_num < deck.rules.max_hands)
        assert self.first_action
        
        self.value = sum(self.cards)
        if self.value > 21:
            self.cards[self.cards.index(11)] = 1
            self.value -= 10
#             print(f'{self.cards=}', f'{self.value=}', f'{self.pair=}', f'{self.to_str()=}')
        
        self.soft = 11 in self.cards
        self.busted = self.value > 21
        self.natural = (self.value == 21) and self.first_action and not self.splitted
        self.surrendered = False
        self.doubled = False
    
    def hit(self, deck, double=False, free=False, count=True):
        card = deck.deal_card(count)
        self.cards.append(card)
        self.value += card
        self.first_action = False
        self.pair = False
        self.doubled = double
        self.free_double += double and free
        self.soft = self.soft or card==11
        while self.value > 21 and self.soft:
            self.cards[self.cards.index(11)] = 1
            self.value -= 10
            self.soft = 11 in self.cards
        if self.value > 21:
            self.busted = True
        return card
    
    def split(self, deck):
        assert self.first_action and self.pair and self.hand_num < deck.rules.max_hands
        act = deck.rules.hit_split_aces or self.to_str()!='AA'
        hand1 = Hand(deck, cards=[self.cards[0]], splitted=True, actionable=act, hand_num=self.hand_num + 1)
        hand2 = Hand(deck, cards=[self.cards[1]], splitted=True, actionable=act, hand_num=self.hand_num, free_hand=True)
        return hand1, hand2
    
    def play_hand(self, deck, dealer, action_table, free_table):
        assert not self.is_dealer
        data = {'player': self.to_str(), 'dealer': dealer.to_str(), 'decks_remaining': deck.decks_remaining()}
        data.update({'hilow_act': deck.true_count().copy()['hilow']})
        
        if self.natural or dealer.natural or self.busted or self.value==21 or not self.actionable:
            deck.count(dealer.cards[0])
            data['action'] = None
            data['payout'] = resolve_hands([self], dealer, deck.rules)
            return data
        
        action = action_table.loc[self.to_str(), dealer.to_str()]
        player_hands = self.play_player(deck, dealer, action_table, free_table)
        
        if not all(hand.busted for hand in player_hands):
            dealer.play_dealer(deck)
        data['action'] = action
        data['payout'] = resolve_hands(player_hands, dealer, deck.rules)
        data['free_bets'] = count_free_bets(player_hands)
        data['dealer_push'] = dealer.push
        data['dealer_bust'] = dealer.busted and not dealer.push
        return data
    
    def play_player(self, deck, dealer, action_table, free_table, action=None):
        if self.natural or dealer.natural or self.busted or self.value==21 or not (self.actionable or (self.to_str()=='AA' and deck.rules.resplit_aces)):
            return [self]
        if action is None and (self.actionable or (self.to_str()=='AA' and deck.rules.resplit_aces)):
            if self.free_hand:
                action = free_table.loc[self.to_str(), dealer.to_str()]
            else:
                action = action_table.loc[self.to_str(), dealer.to_str()]
        
        if action[0] == 'R':
            if self.first_action and deck.rules.allow_surrender and not self.splitted:
                self.surrendered = True
                return [self]
            else: action = action[1].upper()
        if action[0] == 'P':
            if self.to_str()=='AA' and (self.hand_num >= deck.rules.max_hands or (self.splitted and not deck.rules.resplit_aces)):
                action = 'H'
            elif len(action)==1 or deck.rules.double_after_split:
                assert self.first_action and self.pair and self.hand_num < deck.rules.max_hands
                act = deck.rules.hit_split_aces or self.to_str()!='AA'
                hand1 = Hand(deck, cards=[self.cards[0]], splitted=True, actionable=act, hand_num=self.hand_num + 1)
                h1 = hand1.play_player(deck, dealer, action_table, free_table)
                hand2 = Hand(deck, cards=[self.cards[1]], splitted=True, actionable=act, hand_num=self.hand_num + len(h1), free_hand=True)
                h2 = hand2.play_player(deck, dealer, action_table, free_table)
                return h1 + h2
            else: action = action[1].upper()
        if action[0] == 'D':
            if self.first_action and (deck.rules.double_after_split or not self.splitted):
                card = self.hit(deck, double=True)
                return [self]
            elif len(action)==1:
                action = 'H'
            else: action = action[1].upper()
        if action == 'FD':
            if self.first_action:
                card = self.hit(deck, double=True, free=True)
                return [self]
            else:
                action = 'H'
        if action == 'H':
            card = self.hit(deck)
            return self.play_player(deck, dealer, action_table, free_table)
        elif action == 'S':
            return [self]
        
        print(action)
        raise Exception
    
    def play_dealer(self, deck):
        assert self.is_dealer
        deck.count(self.cards[0])
        while self.value < 17 or (deck.rules.hit_soft17 and self.value==17 and self.soft):
            card = self.hit(deck)
        self.push = self.value==22
        return self
    
    def copy(self):
        assert len(self.cards)>=2
        return Hand(None, self.cards, splitted=self.splitted, is_dealer=self.is_dealer, actionable=self.actionable)
        
    def card2str(self, card):
        return {11:'A', 10:'T', 1:'A'}.get(card, str(card))
        
    def to_str(self):
        if self.is_dealer:
            return self.card2str(self.cards[1])
        if self.value == 21:
            return '21'
        if self.pair and self.first_action:
            return self.card2str(self.cards[0])*2
        if self.soft:
            return 'A' + self.card2str(self.value-11)
        return str(self.value)
        
def resolve_hands(player_hands, dealer, rules):
    total = 0
    for hand in player_hands:
        payout = 0
        if hand.natural:
            payout = rules.bj_payout if not dealer.natural else 0
        elif dealer.natural:
            payout = 0 if hand.free_hand else -1
        elif hand.surrendered:
            payout = -.5
        elif hand.busted:
            payout = (0 if hand.free_hand else -1) - (1 if hand.doubled and not hand.free_double else 0)
        elif dealer.push:
            payout = 0
        elif dealer.busted:
            payout = 2 if hand.doubled else 1
        elif hand.value > dealer.value:
            payout = 2 if hand.doubled else 1
        elif hand.value < dealer.value:
            payout = (0 if hand.free_hand else -1) - (1 if hand.doubled and not hand.free_double else 0)
        total += payout
    return total

def count_free_bets(player_hands):
    return sum([hand.free_hand*1 + hand.free_double*1 for hand in player_hands])

class FreeBetBlackjack:
    def __init__(self, bj_payout=1.5, hit_soft17=True, allow_surrender=False, double_after_split=True, resplit_aces=True, hit_split_aces=False):
        self.rules = Rules(bj_payout, hit_soft17, allow_surrender, double_after_split, resplit_aces, hit_split_aces)
        self.action_table = pd.read_csv('/Users/alex/Documents/Gambling/Blackjack/decision_table - FB - 55 split - real money.csv', index_col=0).fillna('')
        self.free_table = pd.read_csv('/Users/alex/Documents/Gambling/Blackjack/decision_table - FB - 55 split - free hand.csv', index_col=0).fillna('')
        
    # play a single deck of cards and record data on each hand
    def play_single_deck(self, num_hands=1):
        num_decks = 1
        self.deck = Deck(self.rules, num_decks)
        self.data = []
        for i in range(6-num_hands):
            payout = self.play_hand(self.deck)
            self.data.append(payout)
        return self.data
            
    def play_multi_deck(self, decks=2, penetration=1.25):
        num_decks = decks
        self.deck = Deck(self.rules, num_decks)
        self.data = []
        while self.deck.decks_remaining() > self.deck.num_decks - penetration:
            payout = self.play_hand(self.deck)
            if type(payout) is list:
                print(payout)
            self.data.append(payout)
        return self.data
        
    # play a hand out of the deck, modify data with results, return deck with cards removed
    def play_hand(self, deck, split_hands=1, player=None, dealer=None):
        count = deck.true_count().copy()['hilow']
        player = Hand(deck)
        dealer = Hand(deck, is_dealer=True)
        
#         player_hands = player.play_player(deck, dealer, self.action_table)
#         dealer = dealer.play_dealer(deck)
        
        data = player.play_hand(deck, dealer, self.action_table, self.free_table)
        data.update({'hilow_bet': count})
        return data


In [50]:
game = FreeBetBlackjack(bj_payout=1.5, hit_soft17=True, allow_surrender=False)
decks = 6
penetration = 4.75


i = 3
max_i = 3
df = []
t = time.time()
while i <= max_i:
    data = game.play_multi_deck(decks, penetration)
    df += data
    if len(df) > 1e7:
        print(i, time.time() - t)
#         print(payouts)
#         print(pd.DataFrame(payouts))
        df = pd.DataFrame(df)
        df.to_csv(f'/Users/alex/Documents/Gambling/Blackjack/6deck_freebet_3to2_H17_55split_{i}.csv')
        i += 1
        t = time.time()
        if i <= max_i:
            df = []
# sum(payouts), len(payouts), sum(payouts)/len(payouts), (sum([p**2 for p in payouts])**.5/len(payouts))

3 505.60248708724976


In [52]:
df

,player,dealer,decks_remaining,hilow_action,action,payout,hilow_bet,free_bets,dealer_push,dealer_bust
0,21,6,5.923077,-0.168831,None,1.5,0.000000,NaN,NaN,NaN
1,11,T,5.846154,-0.171053,FD,0.0,-0.168831,1.0,True,False
2,88,2,5.730769,0.348993,P,0.0,0.172185,1.0,True,False
3,14,7,5.576923,0.179310,H,1.0,0.000000,0.0,False,True
4,8,8,5.461538,0.549296,H,1.0,0.180556,0.0,False,True
...,...,...,...,...,...,...,...,...,...,...
10000004,21,9,1.557692,3.851852,None,1.5,4.894118,NaN,NaN,NaN
10000005,11,A,1.480769,3.376623,None,-1.0,3.209877,NaN,NaN,NaN
10000006,19,T,1.403846,1.424658,S,-1.0,2.701299,0.0,False,False
10000007,A6,T,1.326923,0.000000,H,1.0,0.712329,0.0,False,True


In [55]:
df['payout'].mean()

-0.017403484336864096

In [56]:
pot_of_gold = pd.Series({
    0: -1,
    1: 3,
    2: 12,
    3: 30,
    4: 50,
    5: 100,
    6: 100,
    7: 100,
})
df['free_bets'] = df['free_bets'].fillna(0).astype(int)
df['pot_of_gold'] = pot_of_gold[df['free_bets']].values
df['pot_of_gold'].mean()

-0.09657771308005823

In [57]:
(df.groupby('free_bets')['pot_of_gold'].count() / len(df)).apply(lambda x: '%.8f' % x)

free_bets
0    0.83806755
1    0.14309417
2    0.01492879
3    0.00333840
4    0.00048400
5    0.00007850
6    0.00000790
7    0.00000070
Name: pot_of_gold, dtype: object

In [58]:
pairs = df.loc[(df['player'].str[0]==df['player'].str[1]) & ~df['player'].str[0].isin(['1', 'T'])]
pairs

,player,dealer,decks_remaining,hilow_act,action,payout,hilow_bet,free_bets,dealer_push,dealer_bust,pot_of_gold
2,88,2,5.730769,0.348993,P,0.0,0.172185,1,True,False,3
42,99,9,1.538462,-3.900000,P,-1.0,-3.714286,1,False,False,3
43,55,3,1.384615,-2.888889,P,1.0,-4.789474,1,False,False,3
98,88,5,5.153846,0.000000,P,3.0,-0.191176,2,False,True,12
106,AA,T,4.288462,0.000000,P,-1.0,0.687225,1,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...
9999865,44,4,2.057692,4.373832,P,2.0,2.810811,1,False,True,3
9999869,88,T,1.596154,2.506024,P,1.0,2.988506,1,False,False,3
9999895,88,2,3.673077,0.816754,P,1.0,0.533333,1,False,False,3
9999901,55,9,3.038462,-1.316456,P,-1.0,-1.925926,2,False,False,12


In [59]:
doubles = df.loc[df['player'].isin(['9', '10', '11'])]
doubles

,player,dealer,decks_remaining,hilow_act,action,payout,hilow_bet,free_bets,dealer_push,dealer_bust,pot_of_gold
1,11,T,5.846154,-0.171053,FD,0.0,-0.168831,1,True,False,3
5,11,A,5.346154,0.374101,None,-1.0,0.368794,0,NaN,NaN,-1
17,10,7,4.096154,-0.244131,FD,0.0,-0.479263,1,False,False,3
24,11,2,3.307692,-0.906977,FD,0.0,-1.477273,1,False,False,3
25,10,4,3.192308,-0.626506,FD,-1.0,-1.529412,1,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...
9999996,11,6,2.442308,0.818898,FD,2.0,0.000000,1,False,True,3
10000000,10,3,1.961538,5.098039,FD,2.0,3.433962,1,False,False,3
10000003,10,8,1.673077,4.183908,FD,2.0,3.428571,1,False,True,3
10000005,11,A,1.480769,3.376623,None,-1.0,3.209877,0,NaN,NaN,-1


In [48]:
df

,player,dealer,decks_remaining,hilow,action,payout,free_bets,dealer_push,dealer_bust,pot_of_gold
0,TT,4,5.923077,-0.168831,S,1.0,0,False,True,-1
1,18,6,5.807692,-0.172185,S,-1.0,0,False,False,-1
2,6,7,5.711538,0.175084,H,1.0,0,False,True,-1
3,TT,T,5.576923,-0.896552,S,1.0,0,False,False,-1
4,TT,2,5.500000,-1.090909,S,1.0,0,False,False,-1
...,...,...,...,...,...,...,...,...,...,...
10000007,TT,T,1.653846,-4.837209,S,-1.0,0,False,False,-1
10000008,15,9,1.557692,-3.851852,H,-1.0,0,False,False,-1
10000009,5,T,1.461538,-4.105263,H,1.0,0,False,False,-1
10000010,15,9,1.346154,-3.714286,H,-1.0,0,False,False,-1


In [60]:
(pairs.groupby(['player','free_bets'])['pot_of_gold'].count()).unstack()#.apply(lambda x: '%.8f' % x)

free_bets,0,1,2,3,4,5,6,7
player,,,,,,,,
22,2663.0,28098.0,20472.0,5048.0,786.0,135.0,18.0,NaN
33,2759.0,27793.0,20492.0,5176.0,795.0,126.0,13.0,NaN
44,2801.0,27646.0,20492.0,5091.0,750.0,127.0,13.0,3.0
55,2738.0,34400.0,16010.0,3134.0,437.0,87.0,9.0,2.0
66,2790.0,27566.0,20621.0,5221.0,779.0,113.0,10.0,1.0
77,2696.0,27920.0,20379.0,5039.0,760.0,141.0,14.0,1.0
88,2757.0,34112.0,16302.0,2996.0,387.0,44.0,2.0,NaN
99,2688.0,41892.0,10701.0,1410.0,146.0,12.0,NaN,NaN
AA,2499.0,49983.0,3819.0,269.0,NaN,NaN,NaN,NaN


In [67]:
X = sm.add_constant(df['hilow_bet'])
Y = df['pot_of_gold']
r = sm.OLS(Y, X).fit()
r.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            pot_of_gold   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     8474.
Date:                Sat, 08 Oct 2022   Prob (F-statistic):               0.00
Time:                        01:03:11   Log-Likelihood:            -2.5363e+07
No. Observations:            10000009   AIC:                         5.073e+07
Df Residuals:                10000007   BIC:                         5.073e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1022      0.001   -105.494      0.000      -0.104      -0.100
hilow_bet     -0.0345      0.000    -92.056      0.000      -0.035      -0.034
==============================================================================
Omnibus:                 15230019.350   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      11776469166.958
Skew:                           9.416   Prob(JB):                         0.00
Kurtosis:                     170.060   Cond. No.                         2.59
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
df.loc[df['hilow_bet']<-3, 'pot_of_gold'].mean()

0.07006869234302915

In [81]:
len(df.loc[df['hilow_bet']<-3, 'pot_of_gold'])/len(df)

0.10608150452664593

In [74]:
0.07403640216587985*25

1.8509100541469963

In [79]:
0.07403640216587985*.1060

0.007847858629583265

In [66]:
df['hilow_int'] = df['hilow_bet'].round(0).astype(int)
df.groupby('hilow_int')['pot_of_gold'].mean()

hilow_int
-23   -1.000000
-22   -1.000000
-21    0.200000
-20    2.125000
-19    0.612245
-18    1.326316
-17    0.803279
-16    0.536160
-15    0.681234
-14    0.328255
-13    0.452966
-12    0.445215
-11    0.290440
-10    0.270037
-9     0.232924
-8     0.168285
-7     0.137516
-6     0.110985
-5     0.081927
-4     0.039297
-3    -0.006277
-2    -0.040276
-1    -0.067319
 0    -0.103896
 1    -0.137742
 2    -0.168819
 3    -0.202618
 4    -0.233055
 5    -0.259606
 6    -0.315885
 7    -0.331340
 8    -0.353649
 9    -0.387765
 10   -0.387129
 11   -0.486496
 12   -0.461846
 13   -0.554707
 14   -0.486352
 15   -0.511688
 16   -0.455497
 17   -0.714286
 18   -0.609756
 19    0.050000
 20   -1.000000
 21    0.000000
 22   -1.000000
 23   -1.000000
Name: pot_of_gold, dtype: float64